# Project 1
## Discrete Event Simulations: Loading (and Unloading) an Airplane
### Hiram College: Computer Simulation (CPSC 381)
**Description:** This project will utilize Discrete Event Simulation (DES) to simulate the loading and unloading of a plane to determine the average time for a flight to unload every passenger on a full flight based on the number of rows and seats in the plane.  

In [3]:
import simpy
import numpy as np
import string
import random

class Passenger:
    def __init__(self, id, row: int, seat: str, speed: float):
        self.id = id
        self.row = row
        self.seat = seat
        self.speed = speed
        self.unloaded = False

    def get_data(self):
        return [self.id, self.row, self.seat, self.speed, self.unloaded]

    def get_id(self):
        return self.id

    def get_row(self):
        return self.row

    def get_seat(self):
        return self.seat

    def get_speed(self):
        return self.speed

class Plane:
    def __init__(self, env: simpy.Environment, num_rows: int, num_seats_per_row: int, min_passenger_speed: float = 3, max_passenger_speed: float = 6):
        self.env = env
        self.id = id
        self.num_rows = num_rows
        self.num_seats_per_row = num_seats_per_row
        self.min_passenger_speed = min_passenger_speed
        self.max_passenger_speed = max_passenger_speed
        
        self.loaded_passengers = self._setup_plane()
        self.unloaded_passengers = []

    def _setup_plane(self):
        num_passengers = self.num_rows * self.num_seats_per_row
        uppercase_alphabet = list(string.ascii_uppercase)
        seat_labels = []
        for i in range(self.num_seats_per_row):
            seat_labels.append(uppercase_alphabet[i])

        loaded_passengers = []
        row = 1
        seat = seat_labels[0]
        seat_step = 0
        for i in range(num_passengers):
            seat = seat_labels[seat_step]
            
            # Create new passenger (speed is a random float between min_passenger_speed and max_passenger_speed)
            new_passenger = Passenger(i, row, seat, random.uniform(self.min_passenger_speed, self.max_passenger_speed))
            loaded_passengers.append(new_passenger)
            
            if seat == seat_labels[-1]:
                row += 1
                seat_step = 0
            else:
                seat_step += 1

        return loaded_passengers

    def unload_passenger(self, passenger):
        self.idle = False
        time = passenger.get_row() * 3 / passenger.get_speed()
        yield self.env.timeout(delay=time)
        print(f"Unloading passenger {passenger.id} at {self.env.now/60} minutes.")
        passenger.unloaded = True
        self.idle = True
        return passenger

    def unload_plane(self):
        print("Unloading airplane...")
        while len(self.loaded_passengers) > 0:
            p = self.loaded_passengers[0]
            yield env.process(self.unload_passenger(p))
            self.loaded_passengers.remove(p)
            self.unloaded_passengers.append(p)
        print("Unloading complete.") 

    def print_passenger_data(self):
        print("Number of passengers on plane: " + str(len(self.loaded_passengers)))
        for i in range(len(self.loaded_passengers)):
            passenger = self.loaded_passengers[i]
            print(passenger.get_data())

num_rows = 25
num_seats_per_row = 4
env = simpy.Environment()
plane = Plane(env, num_rows, num_seats_per_row)
plane.print_passenger_data()
env.process(plane.unload_plane())
env.run()

Number of passengers on plane: 100
[0, 1, 'A', 3.8456005860443483, False]
[1, 1, 'B', 5.512890020283714, False]
[2, 1, 'C', 4.279784082090838, False]
[3, 1, 'D', 5.181260714774619, False]
[4, 2, 'A', 4.8073882154604295, False]
[5, 2, 'B', 5.470219878548546, False]
[6, 2, 'C', 3.0108920196726032, False]
[7, 2, 'D', 5.522918505458723, False]
[8, 3, 'A', 5.96337152922151, False]
[9, 3, 'B', 5.449489442575528, False]
[10, 3, 'C', 4.127592268995139, False]
[11, 3, 'D', 5.8913183754333325, False]
[12, 4, 'A', 3.5109651490867164, False]
[13, 4, 'B', 5.605791186900215, False]
[14, 4, 'C', 4.17629402206326, False]
[15, 4, 'D', 3.6805630446129842, False]
[16, 5, 'A', 5.234219641975856, False]
[17, 5, 'B', 3.2238715985029804, False]
[18, 5, 'C', 3.0088402568635555, False]
[19, 5, 'D', 5.706280922804298, False]
[20, 6, 'A', 3.569871326323003, False]
[21, 6, 'B', 5.5389951681838125, False]
[22, 6, 'C', 4.3764966627547945, False]
[23, 6, 'D', 5.760893060482237, False]
[24, 7, 'A', 5.050560604756448,